In [1]:
import numpy as np  # noqa
import pandas as pd  # noqa
from pandas import DataFrame

# --------------------------------
# Add your lib to import here
import talib.abstract as ta
from datetime import datetime, timedelta, timezone
import math
from enum import Enum
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler , MinMaxScaler , RobustScaler
import joblib
from sklearn.decomposition import PCA

In [9]:
import ccxt.pro as ccxt
import pandas as pd
import re

# Initialize the exchange
exchange = ccxt.binance()  # Example exchange, replace with your desired exchange
exchange.set_sandbox_mode(True)

exchange.api = "52a65ac457d28e61c7f76a769754b3bdfe413d7c62e09a8605b55aaa140e9284"
exchange.apiKey = "52a65ac457d28e61c7f76a769754b3bdfe413d7c62e09a8605b55aaa140e9284"
exchange.secret = "c86f35f7b8e4f75154ef39c650ff1513657a67799a51ee7429dd4d64939ba386"
leverage = 20
exchange.options["defaultType"] = "future"
# Define symbol and timeframe
symbol = 'BTC/USDT:USDT'  # Example symbol
timeframe = '1m'  # Example timeframe
await exchange.set_leverage(leverage=leverage, symbol=symbol)

# Initialize an empty DataFrame to store the candles
dataframe = pd.DataFrame()

# Fetch 10,000 candles in chunks of 1500 candles
num_candles_to_fetch = 2000
candles_per_request = 1000
start_index = 0
current_time = datetime.strptime(str(datetime.utcnow()).split('.')[0],'%Y-%m-%d %H:%M:%S')

markets = await exchange.load_markets()
tick_size = float(markets[symbol]['info']['filters'][0]['tickSize'])
while start_index < num_candles_to_fetch:
    # Determine the end index for this request
    end_index = min(start_index + candles_per_request, num_candles_to_fetch)

    # minute_data = (num_candles_to_fetch / end_index)*candles_per_request
    minute_data = num_candles_to_fetch - start_index

    timeframe_int = int(re.search(r'\d+', str(timeframe)).group())
    previous_time_utc = str(current_time - timedelta(minutes=minute_data*timeframe_int))
    formatted_datetime_str = previous_time_utc.split('.')[0]
    since = exchange.parse8601(formatted_datetime_str)
    # round(datetime.strptime(formatted_datetime_str, '%Y-%m-%d %H:%M:%S').timestamp())
    # if (end_index == (num_candles_to_fetch-candles_per_request)) else None
    # Fetch candles for this chunk
    candles = await exchange.fetch_ohlcv(
        symbol,
        timeframe, 
        limit=candles_per_request, 
        since=since,
        params={'price': 'mark'}
    )
    
    # Convert candles to DataFrame
    candles_df = pd.DataFrame(candles, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    candles_df['date'] = pd.to_datetime(candles_df['date'], unit='ms')
    # Append the candles to the DataFrame
    dataframe = pd.concat([dataframe, candles_df], ignore_index=True)
    
    # # Update start index for the next request
    start_index += candles_per_request

print(dataframe)

binance requires to release all resources with an explicit call to the .close() coroutine. If you are using the exchange instance with async coroutines, add `await exchange.close()` to your code into a place when you're done with the exchange and don't need the exchange instance anymore (at the end of your async coroutine).
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x727d6508a750>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x727d2ef0c890>, 26131.798590309)]', '[(<aiohttp.client_proto.ResponseHandler object at 0x727d2ef0c350>, 26131.864894559), (<aiohttp.client_proto.ResponseHandler object at 0x727d2ef0d250>, 26133.357740267)]']
connector: <aiohttp.connector.TCPConnector object at 0x727d646252e0>
/tmp/ipykernel_150389/3340177743.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.d

                    date          open          high           low  \
0    2024-10-21 00:14:00  69109.419149  69119.224681  69108.840851   
1    2024-10-21 00:15:00  69121.598511  69194.610000  69121.598511   
2    2024-10-21 00:16:00  69194.385106  69284.452766  69189.399574   
3    2024-10-21 00:17:00  69283.895957  69406.002766  69283.895957   
4    2024-10-21 00:18:00  69360.375957  69381.569362  69328.728936   
...                  ...           ...           ...           ...   
1995 2024-10-22 09:29:00  66862.418085  66864.383830  66860.604468   
1996 2024-10-22 09:30:00  66863.453404  66884.623404  66837.713191   
1997 2024-10-22 09:31:00  66830.498298  66893.219787  66826.384468   
1998 2024-10-22 09:32:00  66892.883191  66892.883191  66803.428936   
1999 2024-10-22 09:33:00  66812.640638  66868.542553  66812.640638   

             close  volume  
0     69119.207021     0.0  
1     69194.498085     0.0  
2     69284.452766     0.0  
3     69365.897872     0.0  
4     69374.22

In [ ]:
def calculate_CCI(dataframe,index,diff,ma,p):
    close_array = dataframe['open'].to_numpy()
    close_array = close_array[:index.name]
    diff = diff.to_numpy()
    diff = diff[index.name]
    ma = ma.to_numpy()
    ma = ma[index.name]

    if(len(close_array) < p-1):
        return np.nan

    s = 0

    for i in range(len(close_array),len(close_array)-p,-1):
        s = s + abs(dataframe['open'][i] - ma)
    mad = s / p

    mcci = diff/mad/0.015
    
    return mcci

def calculate_cci_improved(source1:pd.Series,source2:pd.Series,source3:pd.Series,length) -> pd.Series:
    source1:np.ndarray = source1.to_numpy()
    source2:np.ndarray = source2.to_numpy()
    source3:np.ndarray = source3.to_numpy()
    
    windows = np.lib.stride_tricks.sliding_window_view(source1, window_shape=(length,))
    source3_modify = source3[:, np.newaxis][length-1:]
    sums = np.sum(np.abs(windows - source3_modify),axis=1)
        
    mad = sums / length

    mad_series = pd.Series(index=range(len(source1)), dtype=float)
    mad_series[-len(mad):] = mad
    mad_series = mad_series.to_numpy()
    mcci = source2/mad_series/0.015
    return mcci


def rescale(src, old_min, old_max, new_min, new_max):
    return new_min + (new_max - new_min) * (src - old_min) / np.maximum((old_max - old_min), 10e-10)

def normalize(value, min_val, max_val,df):
    index = value.name
    src = pd.Series(df[:index+1])

    historic_min = 10e10
    historic_max = -10e10

    src_filled_min = src.fillna(historic_min)
    src_filled_max = src.fillna(historic_max)
    historic_min = min(src_filled_min.min(), historic_min) if not pd.isna(src_filled_min.min()) else historic_min
    historic_max = max(src_filled_max.max(), historic_max) if not pd.isna(src_filled_max.max()) else historic_max

    normalized_src = (min_val + (max_val - min_val) * (src[index] - historic_min)) / max((historic_max - historic_min), 10e-10)
    return normalized_src

def normalize_optimized(min_val, max_val,source:pd.Series):
    source = source.to_numpy()
    historic_min = 10e10
    historic_max = -10e10
    src_filled_min = np.nan_to_num(source,historic_min) #source.fillna(historic_min)
    src_filled_max = np.nan_to_num(source,historic_max) #source.fillna(historic_max)
    historic_min = np.minimum.accumulate(src_filled_min)
    historic_max = np.maximum.accumulate(src_filled_max)
    division_value = np.maximum((historic_max - historic_min),10e-10)
    normalized_src = (min_val + (max_val - min_val) * (source - historic_min)) / division_value
    return normalized_src

def n_rsi(src, n1, n2):
    rsi = ta.RSI(src,n1)
    ema_rsi = ta.EMA(rsi,n2)
    return rescale(ema_rsi, 0, 100, 0, 1)

def n_cci(dataframe, n1 , n2):
    df = dataframe.copy()
    source = df['close']

    df['mas'] = ta.SMA(source,n1)
    df['diffs'] = source - df['mas']

    # df['cci'] = df.apply((lambda index: calculate_CCI(df,index,df['diffs'],df['mas'],n1)),axis=1)
    df['cci'] = pd.Series(calculate_cci_improved(dataframe['open'],df['diffs'],df['mas'],n1))

    df['ema_cci'] = ta.EMA(df['cci'],n2)

    # normalized_wt_diff = df.apply((lambda x : normalize(x,0,1,df['ema_cci'])),axis=1)
    normalized_wt_diff = pd.Series(normalize_optimized(0,1,df['ema_cci']))
    return normalized_wt_diff

def n_wt(src, n1, n2):
    ema1 = ta.EMA(src, n1)
    ema2 = ta.EMA(np.abs(src - ema1), n1)
    ci = (src - ema1) / (0.015 * ema2)
    wt1 = ta.EMA(ci, n2)
    wt2 = ta.SMA(wt1, 4)
    diff = wt1 - wt2
    # normalized_wt_diff = df.apply((lambda x : normalize(x,0,1,diff)),axis=1)
    normalized_wt_diff = pd.Series(normalize_optimized(0,1,pd.Series(diff)))
    return normalized_wt_diff

def calculate_tr_optimized(high,low,close):
    previos_close = np.roll(close,1)

    diff_h_n_l = high - low
    abs_value_h_n_c = np.abs(high - previos_close)
    abs_value_h_n_c[0] = abs(high[0] - 0)
    abs_value_l_n_c = np.abs(low - previos_close)
    abs_value_l_n_c[0] = abs(low[0] - 0)
    tr = np.maximum(np.maximum(diff_h_n_l,abs_value_h_n_c),abs_value_l_n_c)
    return tr

def calculate_directionalMovementPlus_optimized(high,low):
    prev_high = np.roll(high,1)
    prev_low = np.roll(low,1)

    diff_h_n_ph = high - prev_high
    diff_h_n_ph[0] = high[0] - 0
    diff_pl_n_l = prev_low - low
    diff_pl_n_l[0] = 0 - low[0]
    dmp_value = np.maximum(diff_h_n_ph, 0) * (diff_h_n_ph > diff_pl_n_l)
    return dmp_value

def calculate_negMovement_optimized(high,low):
    prev_high = np.roll(high,1)
    prev_low = np.roll(low,1)

    diff_h_n_ph = high - prev_high
    diff_h_n_ph[0] = high[0] - 0
    diff_pl_n_l = prev_low - low
    diff_pl_n_l[0] = 0 - low[0]
    negMovement = np.maximum(diff_pl_n_l, 0) * (diff_pl_n_l > diff_h_n_ph)
    return negMovement

def n_adx_optimized(highSrc:pd.Series, lowSrc:pd.Series, closeSrc:pd.Series, n1:int):
    length = n1
    highSrc_numpy = highSrc.to_numpy()
    lowSrc_numpy = lowSrc.to_numpy()
    closeSrc_numpy = closeSrc.to_numpy()
    
    tr = calculate_tr_optimized(highSrc_numpy,lowSrc_numpy,closeSrc_numpy)
    directionalMovementPlus = calculate_directionalMovementPlus_optimized(highSrc_numpy,lowSrc_numpy)
    negMovement = calculate_negMovement_optimized(highSrc_numpy,lowSrc_numpy)

    trSmooth = np.zeros_like(closeSrc_numpy)
    trSmooth[0] = np.nan
    for i in range(0, len(tr)):
        trSmooth[i] = trSmooth[i-1] - trSmooth[i-1] / length + tr[i]

    smoothDirectionalMovementPlus = np.zeros_like(closeSrc)
    smoothDirectionalMovementPlus[0] = np.nan
    for i in range(0, len(directionalMovementPlus)):
        smoothDirectionalMovementPlus[i] = smoothDirectionalMovementPlus[i-1] - smoothDirectionalMovementPlus[i-1] / length + directionalMovementPlus[i]

    smoothnegMovement = np.zeros_like(closeSrc)
    smoothnegMovement[0] = np.nan
    for i in range(0, len(negMovement)):
        smoothnegMovement[i] = smoothnegMovement[i-1] - smoothnegMovement[i-1] / length + negMovement[i]

    diPositive = smoothDirectionalMovementPlus / trSmooth * 100
    diNegative = smoothnegMovement / trSmooth * 100
    dx = np.abs(diPositive - diNegative) / (diPositive + diNegative) * 100
    dx_series = pd.Series(dx)

    adx = dx_series.copy()
    adx.iloc[:length] = adx.rolling(length).mean().iloc[:length]
    adx = adx.ewm(alpha=(1.0/length),adjust=False).mean()
    return rescale(adx, 0, 100, 0, 1)

PEAK, VALLEY = 1, -1
def _identify_initial_pivot(X, up_thresh, down_thresh):
    x_0 = X[0]
    max_x = x_0
    max_t = 0
    min_x = x_0
    min_t = 0
    up_thresh += 1
    down_thresh += 1

    for t in range(1, len(X)):
        x_t = X[t]

        if x_t / min_x >= up_thresh:
            return VALLEY if min_t == 0 else PEAK

        if x_t / max_x <= down_thresh:
            return PEAK if max_t == 0 else VALLEY

        if x_t > max_x:
            max_x = x_t
            max_t = t

        if x_t < min_x:
            min_x = x_t
            min_t = t

    t_n = len(X)-1
    return VALLEY if x_0 < X[t_n] else PEAK

def peak_valley_pivots_candlestick(close, high, low, up_thresh, down_thresh):
    if down_thresh > 0:
        raise ValueError('The down_thresh must be negative.')

    initial_pivot = _identify_initial_pivot(close, up_thresh, down_thresh)

    t_n = len(close)
    pivots = np.zeros(t_n, dtype='i1')
    pivots[0] = initial_pivot

    up_thresh += 1
    down_thresh += 1

    trend = -initial_pivot
    last_pivot_t = 0
    last_pivot_x = close[0]
    for t in range(1, len(close)):

        if trend == -1:
            x = low[t]
            r = x / last_pivot_x
            if r >= up_thresh:
                pivots[last_pivot_t] = trend#
                trend = 1
                last_pivot_x = high[t]
                last_pivot_t = t
            elif x < last_pivot_x:
                last_pivot_x = x
                last_pivot_t = t
        else:
            x = high[t]
            r = x / last_pivot_x
            if r <= down_thresh:
                pivots[last_pivot_t] = trend
                trend = -1
                last_pivot_x = low[t]
                last_pivot_t = t
            elif x > last_pivot_x:
                last_pivot_x = x
                last_pivot_t = t


    if last_pivot_t == t_n-1:
        pivots[last_pivot_t] = trend
    elif pivots[t_n-1] == 0:
        pivots[t_n-1] = trend

    return pivots

def my_pivothigh(series, left_bars, right_bars):
    # pivot_high = None
    pivot_range = left_bars + right_bars
    # left_edge_value = series.shift(pivot_range)
    
    possible_pivot_high = series[right_bars:].shift(right_bars)
    possible_pivot_high = np.concatenate([np.full(right_bars, np.nan), possible_pivot_high])

    rolling_windows = np.lib.stride_tricks.sliding_window_view(series, window_shape=(pivot_range+1,))
    max_previous_index = np.argmax(rolling_windows, axis=1)

    max_previous_index = np.concatenate([np.full(pivot_range, np.nan), max_previous_index])
    pivot_high_right_bars = pd.Series((pivot_range+1) - max_previous_index - 1)
    condition = pivot_high_right_bars == left_bars
    # result_series = pd.Series(np.where(condition, possible_pivot_high, 0))
    result_series = pd.Series(np.where(condition, -1, 0))
    result_series[result_series != -1] = 0
    return result_series

def my_pivotlow(series, left_bars, right_bars):
    # pivot_low = None
    pivot_range = left_bars + right_bars
    # left_edge_value = series.shift(pivot_range)

    possible_pivot_low = series[right_bars:].shift(right_bars)
    possible_pivot_low = np.concatenate([np.full(right_bars, np.nan), possible_pivot_low])

    rolling_windows = np.lib.stride_tricks.sliding_window_view(series, window_shape=(pivot_range+1,))
    min_previous_index = np.argmin(rolling_windows, axis=1)

    min_previous_index = np.concatenate([np.full(pivot_range, np.nan), min_previous_index])
    pivot_low_right_bars = pd.Series((pivot_range+1) - min_previous_index - 1)
    condition = pivot_low_right_bars == left_bars
    # result_series = pd.Series(np.where(condition, possible_pivot_low, 0))
    result_series = pd.Series(np.where(condition, 1, 0))
    result_series[result_series != 1] = 0
    return result_series

def calculate_up_or_down(pivot_high,pivot_low,pivot_length):
    high = pivot_high.copy()
    low = pivot_low.copy()
    high[high != -1] = 0
    low[low != 1] = 0
    result = high + low
    result[result == 0] = np.nan
    result = result.ffill()
    result[len(result)-pivot_length:] = np.nan
    return result


def highestvalue(_src,_len):
    rolling_windows = np.lib.stride_tricks.sliding_window_view(_src, window_shape=(_len,))
    max_previous_value = np.max(rolling_windows, axis=1)
    max_previous_value = np.concatenate([np.full(_len-1, np.nan), max_previous_value])
    return max_previous_value

def lowestvalue(_src,_len):
    rolling_windows = np.lib.stride_tricks.sliding_window_view(_src, window_shape=(_len,))
    max_previous_value = np.min(rolling_windows, axis=1)
    max_previous_value = np.concatenate([np.full(_len-1, np.nan), max_previous_value])
    return max_previous_value

def change_occurred(arr):
    differences = np.diff(arr)
    differences = np.concatenate([np.full(1, 0), differences])
    return differences != 0

@jit()
def set_h_l_value(high:np.ndarray,low:np.ndarray,direction:np.ndarray):
    price_now_1 = low[0]
    price_now_2 = high[0]
    price_now = low[0]

    price_index = 0
    price_index_1 = 0
    price_index_2 = 0

    price_index_array_1 = np.zeros_like(high)
    price_index_array_2 = np.zeros_like(high)
    price_index_array = np.zeros_like(high)

    for i in range(len(high)):
        if direction[i] != direction[i-1]:
            price_now_1 = price_now_2
            price_index_1 = price_index_2
            price_index_array_1[i] = price_index_1
            price_now_2 = price_now
            price_index_2 = price_index
            price_index_array_2[i] = price_index_2

        if direction[i] > 0:
            if (high[i] > price_now_2):
                price_now_2 = high[i]
                price_index_2 = i
                price_index_array_2[i] = price_index_2
                price_now = low[i]
                price_index = i
                price_index_array[i] = price_index

            if (low[i] < price_now):
                price_now = low[i]
                price_index = i
                price_index_array[i] = price_index

        if direction[i] < 0:
            if (low[i] < price_now_2):
                price_now_2 = low[i]
                price_index_2 = i
                price_index_array_2[i] = price_index_2
                price_now = high[i]
                price_index = i
                price_index_array[i] = price_index

            if (high[i] > price_now):
                price_now = high[i]
                price_index = i
                price_index_array[i] = price_index
    
    return price_index_array_1,price_index_array_2

@jit()
def findBarSince(src:np.ndarray):
    last_true_index = None
    index_distance_array = np.zeros_like(src).astype(np.int64)
    counter = 0
    for i, _ in enumerate(src):
        if src[i]:
            last_true_index = i
            counter = 0
        elif last_true_index is not None:
            if i > last_true_index:
                counter = counter + 1
                index_distance_array[i] = counter
    return index_distance_array

@jit()
def find_ytrain(z1:np.ndarray,z2:np.ndarray,direction:np.ndarray,high:np.ndarray,low:np.ndarray):
    cutted_z1 = z1[direction != np.roll(direction,1)]
    cutted_z2 = z2[direction != np.roll(direction,1)]
    cutted_direction = direction[direction != np.roll(direction,1)]

    iterator = zip(cutted_z1,cutted_direction)
    result = np.zeros_like(direction)
    previous_index = 0
    for i,value in enumerate(iterator):
        result[previous_index:int(value[0])+1] = value[1]
        
        previous_index = int(value[0])+1

    while previous_index < len(z1)-1:
        data_index_remainder = previous_index
        result_remainder = result[data_index_remainder:] 
        direction_remainder = direction[data_index_remainder:]
        high_remainder = high[data_index_remainder:]
        low_remainder = low[data_index_remainder:]

        max_index = np.argmax(high_remainder)+1
        min_index = np.argmin(low_remainder)+1

        if(result[data_index_remainder-1] == 1):
            result_remainder[:max_index] = -1
            result_remainder[max_index:] = 0
            result_remainder = np.roll(result_remainder,1)
            result_remainder[0] = -1
            result[data_index_remainder:]  = result_remainder
            previous_index = data_index_remainder + max_index
        if(result[data_index_remainder-1] == -1):
            result_remainder[:min_index] = 1   
            result_remainder[min_index:] = 0
            result_remainder = np.roll(result_remainder,1)
            result_remainder[0] = 1
            result[data_index_remainder:]  = result_remainder
            previous_index = data_index_remainder + min_index
        

    # 1 into -1 and -1 into 1 Conversion
    mask_1 = result == 1
    mask_minus1 = result == -1
    result[mask_1] = -1
    result[mask_minus1] = 1
    return result

def zigzagpp(_high,_low,depth,deviation,backstep):
    df = pd.DataFrame()
    df['high'] = pd.Series(_high)
    df['low'] = pd.Series(_low)

    df['highest'] = highestvalue(df['high'],depth)
    hr_condition = np.logical_not(((df['highest'] - df['high']) > (deviation * tick_size)).shift(1))
    hr_condition = np.array(hr_condition, dtype=np.float64)
    df['hr'] = findBarSince(hr_condition)

    df['lowest'] = lowestvalue(df['low'],depth)
    lr_condition = np.logical_not(((df['low'] - df['lowest']) > (deviation * tick_size)).shift(1))
    lr_condition = np.array(lr_condition, dtype=np.float64)
    df['lr'] = findBarSince(lr_condition)

    difference_of_hr_lr_condition = np.logical_not(df['hr'] > df['lr'])
    difference_of_hr_lr_condition = np.array(difference_of_hr_lr_condition,dtype=np.bool_)
    difference_of_hr_lr = findBarSince(difference_of_hr_lr_condition) >= backstep
    df['direction'] = np.where(difference_of_hr_lr,-1,1)

    price_index_1 , price_index_2 = set_h_l_value(df['high'].values,df['low'].values,df['direction'].values)

    price_index_1 = pd.Series(price_index_1)
    price_index_2 = pd.Series(price_index_2)

    return price_index_1,price_index_2,df['direction']

def adaptiveTrendFinder(close: np.ndarray,periods:np.ndarray):
    logSource = np.log(close)
    trend_direction = np.zeros_like(close,dtype=np.float64)
    detectedPeriod = np.zeros_like(close,dtype=np.float64)
    highestPearsonR = np.zeros_like(close,dtype=np.float64)
    for idx, current_value in enumerate(close):
        result = calculate_trend_direction(idx, close ,logSource, periods)
        trend_direction[idx] = result[0]
        detectedPeriod[idx] = result[1]
        highestPearsonR[idx] = result[2]
    return trend_direction,detectedPeriod,highestPearsonR

def calculate_trend_direction(x:int, close:np.ndarray,logSource:np.ndarray, periods:np.ndarray):
    # if(x == len(dataframe)-1) | (x.name == len(dataframe)-2):
    if (x >= periods[2]):
        devMultiplier = 2.0
        # Calculate Deviation,PersionR,Slope,Intercept
        stdDev01, pearsonR01, slope01, intercept01 = calcDev(
            periods[1], close, logSource , x)
        stdDev02, pearsonR02, slope02, intercept02 = calcDev(
            periods[2], close, logSource , x)
        stdDev03, pearsonR03, slope03, intercept03 = calcDev(
            periods[3], close, logSource , x)
        stdDev04, pearsonR04, slope04, intercept04 = calcDev(
            periods[4], close, logSource , x)
        # Find the highest Pearson's R
        highestPearsonR = max(pearsonR01, pearsonR02,pearsonR03,pearsonR04)

        # Determine selected length, slope, intercept, and deviations
        detectedPeriod = 0
        detectedSlope = 0
        detectedIntrcpt = 0
        detectedStdDev = 0

        if highestPearsonR == pearsonR01:
            detectedPeriod = periods[1]
            detectedSlope = slope01
            detectedIntrcpt = intercept01
            detectedStdDev = stdDev01
        elif highestPearsonR == pearsonR02:
            detectedPeriod = periods[2]
            detectedSlope = slope02
            detectedIntrcpt = intercept02
            detectedStdDev = stdDev02
        elif highestPearsonR == pearsonR03:
            detectedPeriod = periods[3]
            detectedSlope = slope03
            detectedIntrcpt = intercept03
            detectedStdDev = stdDev03
        elif highestPearsonR == pearsonR04:
            detectedPeriod = periods[4]
            detectedSlope = slope04
            detectedIntrcpt = intercept04
            detectedStdDev = stdDev04
        else:
            # Default case
            raise Exception(f"Cannot Find Highest PearsonR")

        # Calculate start and end price based on detected slope and intercept
        startPrice = math.exp(
            detectedIntrcpt + detectedSlope * (detectedPeriod - 1))
        endPrice = math.exp(detectedIntrcpt)

        trend_direction = endPrice - startPrice
        return (trend_direction, detectedPeriod, highestPearsonR)
    return (0, 0, 0)

@jit()
def calcDev(length: int, close: np.ndarray, logSource: np.ndarray, index: int):
    period_1 = length - 1
    sumX = 0.0
    sumXX = 0.0
    sumYX = 0.0
    sumY = 0.0
    for i in range(1, length+1):
        lSrc = logSource[index+1-i]
        sumX += i
        sumXX += i * i
        sumYX += i * lSrc
        sumY += lSrc

    slope = np.nan_to_num((length * sumYX - sumX * sumY) /
                          (length * sumXX - sumX * sumX))
    average = sumY / length
    intercept = average - (slope * sumX / length) + slope
    sumDev = 0.0
    sumDxx = 0.0
    sumDyy = 0.0
    sumDyx = 0.0
    regres = intercept + slope * period_1 * 0.5
    sumSlp = intercept

    for i in range(1, period_1+1):
        lSrc = logSource[index+1-i]
        dxt = lSrc - average
        dyt = sumSlp - regres
        lSrc -= sumSlp
        sumSlp += slope
        sumDxx += dxt * dxt
        sumDyy += dyt * dyt
        sumDyx += dxt * dyt
        sumDev += lSrc * lSrc

    unStdDev = math.sqrt(sumDev / period_1)
    divisor = sumDxx * sumDyy
    if divisor == 0 or np.isnan(divisor):
        pearsonR = 0  # Set Pearson correlation coefficient to NaN
    else:
        pearsonR = sumDyx / math.sqrt(divisor)

    return unStdDev, pearsonR, slope, intercept


def calculateSTCIndicator(dataframe, length, fastLength, slowLength):
    df = dataframe.copy()
    EEEEEE = length
    BBBB = fastLength
    BBBBB = slowLength
    # dataframe = heikinashi(dataframe)
    mAAAAA = AAAAA(df, EEEEEE, BBBB, BBBBB)
    return mAAAAA


def AAAA(BBB, BBBB, BBBBB):
    fastMA = ta.EMA(BBB, timeperiod=BBBB)
    slowMA = ta.EMA(BBB, timeperiod=BBBBB)
    AAAA = fastMA - slowMA
    return AAAA

@jit()
def formula_of_AAAAA(output:np.ndarray,input:np.ndarray,length:int,AAA:int):
    for i in range(0, length):
        if (i > 0):
            output[i] = output[i-1] + (AAA * (input[i] - output[i-1]))

    return output

def AAAAA(dataframe, EEEEEE, BBBB, BBBBB):

    AAA = 0.5
    dataframe['DDD'] = 0.0
    dataframe['CCCCC'] = 0
    dataframe['DDDDDD'] = 0
    dataframe['EEEEE'] = 0.0

    dataframe['BBBBBB'] = AAAA(dataframe['close'], BBBB, BBBBB)
    dataframe['CCC'] = dataframe['BBBBBB'].rolling(window=EEEEEE).min()
    dataframe['CCCC'] = dataframe['BBBBBB'].rolling(
        window=EEEEEE).max() - dataframe['CCC']
    dataframe['CCCCC'] = np.where(dataframe['CCCC'] > 0, (dataframe['BBBBBB'] -
                                  dataframe['CCC']) / dataframe['CCCC'] * 100, dataframe['CCCCC'].shift(1).fillna(0))
    
    dataframe['DDD'] = formula_of_AAAAA(dataframe['DDD'].values,dataframe['CCCCC'].values,len(dataframe),AAA)

    dataframe['DDDD'] = dataframe['DDD'].rolling(window=EEEEEE).min()
    dataframe['DDDDD'] = dataframe['DDD'].rolling(
        window=EEEEEE).max() - dataframe['DDDD']
    dataframe['DDDDDD'] = np.where(dataframe['DDD'] > 0, (dataframe['DDD'] - dataframe['DDDD']) /
                                   dataframe['DDDDD'] * 100, dataframe['DDD'].fillna(dataframe['DDDDDD'].shift(1)))

    dataframe['EEEEE'] = formula_of_AAAAA(dataframe['EEEEE'].values,dataframe['DDDDDD'].values,len(dataframe),AAA)

    return dataframe['EEEEE']

In [ ]:
def heikinashi(df: pd.DataFrame) -> pd.DataFrame:
    df_HA = df.copy()
    df_HA['close'] = (df_HA['open'] + df_HA['high'] +
                      df_HA['low'] + df_HA['close']) / 4

    for i in range(0, len(df_HA)):
        if i == 0:
            df_HA.loc[i, 'open'] = (
                (df_HA.loc[i, 'open'] + df_HA.loc[i, 'close']) / 2)
        else:
            df_HA.loc[i, 'open'] = (
                (df_HA.loc[i-1, 'open'] + df_HA.loc[i-1, 'close']) / 2)

    df_HA['high'] = df_HA[['open', 'close', 'high']].max(axis=1)
    df_HA['low'] = df_HA[['open', 'close', 'low']].min(axis=1)

    return df_HA


class FeatureName(Enum):
    rsi = "RSI"
    wt = "WT"
    cci = "CCI"
    adx = "ADX"
    ema = "EMA"
    sma = "SMA"
    macd = "MACD"
    tema = "TEMA"
    open = "OPEN"
    high = "HIGH"
    low = "LOW"
    close = "CLOSE"
    volume = "VOLUME"
    all = "ALL"
    trend = "TREND"
    stc = "STC"

def chooseFeatureName(name: FeatureName, dataframe, paramsA, paramsB):
    df = dataframe.copy()
    source = df['close']
    hlc3 = (df['high'] + df['low'] + df['close']) / 3

    if (name.name == FeatureName.open.name):
        return df['open']
    if (name.name == FeatureName.high.name):
        return df['high']
    if (name.name == FeatureName.low.name):
        return df['low']
    if (name.name == FeatureName.close.name):
        return df['close']
    if (name.name == FeatureName.volume.name):
        return df['volume']
    if (name.name == FeatureName.rsi.name):
        return n_rsi(source, paramsA, paramsB)
    if (name.name == FeatureName.wt.name):
        return n_wt(hlc3, paramsA, paramsB)
    if (name.name == FeatureName.cci.name):
        return n_cci(df, paramsA, paramsB)
    if (name.name == FeatureName.adx.name):
        return n_adx_optimized(df['high'], df['low'], source,paramsA)
    if (name.name == FeatureName.ema.name):
        ema = ta.EMA(df,paramsA) - ta.EMA(df,paramsB)
        old_min = ema.min()
        old_max = ema.max()
        return rescale(ema,old_min,old_max,0,1)
    if (name.name == FeatureName.sma.name):
        sma = ta.SMA(df,paramsA) - ta.SMA(df,paramsB)
        old_min = sma.min()
        old_max = sma.max()
        return rescale(sma,old_min,old_max,0,1)
    if (name.name == FeatureName.macd.name):
        macd = ta.MACD(df)['macdhist']
        old_min = macd.min()
        old_max = macd.max()
        return rescale(macd,old_min,old_max,0,1) 
    if (name.name == FeatureName.tema.name):
        tema = ta.TEMA(df,paramsA) - ta.TEMA(df,paramsB)
        old_min = tema.min()
        old_max = tema.max()
        return rescale(tema,old_min,old_max,0,1)
    if (name.name == FeatureName.all.name):
        all_value = n_rsi(source, paramsA, paramsB) + n_wt(hlc3, paramsA, paramsB) + n_cci(df, paramsA, paramsB) +n_adx_optimized(df['high'], df['low'], source,paramsA)
        old_min = all_value.min()
        old_max = all_value.max()
        return rescale(all_value,old_min,old_max,0,1)
    if (name.name == FeatureName.trend.name):
        df['trend_direction'],df['detectedPeriod'],df['highestPearsonR'] = adaptiveTrendFinder(source.values,paramsA)
        return df['trend_direction']+df['highestPearsonR']
    if (name.name == FeatureName.stc.name):
        fast_length = int(paramsA.split("-")[0])
        slow_length = int(paramsA.split("-")[1])
        length = paramsB
        return calculateSTCIndicator(dataframe,length,fast_length,slow_length)

def compare_value_improved(dataframe:pd.DataFrame,future_count:int):
    df = dataframe.copy()
    # BTC
    # pivots = peak_valley_pivots_candlestick(df['close'], df['high'], df['low'] ,.008,-.008)
    # 1000BONK
    pivots = peak_valley_pivots_candlestick(df['close'], df['high'], df['low'] ,.02,-.02)

    df['Pivots'] = pivots
    df['y_train'] = df['Pivots'].replace(0, np.nan)
    df['y_train'] = df['y_train'].ffill()
    df['y_train'] = df['y_train'].fillna(0)
    return df['y_train']



def extract_features(dataframe: pd.DataFrame, training_params):
    df = dataframe.copy()
    future_count = training_params['future_count']
    feature_count = training_params['feature_count']
    pivot_depth = training_params['pivot_params']['depth']
    pivot_deviation = training_params['pivot_params']['deviation']
    pivot_backstep = training_params['pivot_params']['backstep']

    # f1_name = training_params['f1']['name']
    # f1_param_A = training_params['f1']['paramsA']
    # f1_param_B = training_params['f1']['paramsB']

    # f2_name = training_params['f2']['name']
    # f2_param_A = training_params['f2']['paramsA']
    # f2_param_B = training_params['f2']['paramsB']

    # f3_name = training_params['f3']['name']
    # f3_param_A = training_params['f3']['paramsA']
    # f3_param_B = training_params['f3']['paramsB']

    # f4_name = training_params['f4']['name']
    # f4_param_A = training_params['f4']['paramsA']
    # f4_param_B = training_params['f4']['paramsB']

    # f5_name = training_params['f5']['name']
    # f5_param_A = training_params['f5']['paramsA']
    # f5_param_B = training_params['f5']['paramsB']
    for i in range(1,feature_count+1):
        df[f'f{i}'] = chooseFeatureName(training_params[f'f{i}']['name'], df, training_params[f'f{i}']['paramsA'], training_params[f'f{i}']['paramsB'])

    # df['pivots_high'] = pd.Series(my_pivothigh(df['high'],pivot_params,pivot_params)).shift(-pivot_params)
    # df['pivots_low'] = pd.Series(my_pivotlow(df['low'],pivot_params,pivot_params)).shift(-pivot_params)
    # df['y_train'] = calculate_up_or_down(df['pivots_high'],df['pivots_low'],pivot_params)
    # df['y_train'] = compare_value_improved(df,future_count)
    # df['y_train'] = (df_shifted['close'] < dataframe['close']).astype(int)
    # df['y_train'] = df['y_train'].shift(-future_count)
    # df.loc[len(df)-(future_count):,['y_train']] = np.nan

    df['z1'],df['z2'],df['direction'] = zigzagpp(df['high'],df['low'],pivot_depth,pivot_deviation,pivot_backstep)
    if(future_count > 0):
        df['y_train'] = pd.Series(find_ytrain(df['z1'].values,df['z2'].values,df['direction'].values,df['high'].values,df['low'].values)).shift(-future_count)
    else:
        df['y_train'] = pd.Series(find_ytrain(df['z1'].values,df['z2'].values,df['direction'].values,df['high'].values,df['low'].values))
    df.loc[df['y_train'] == 0,['y_train']] = 0
    return df

In [ ]:
def filter_volatility(dataframe:pd.DataFrame,minLength:int,maxLength:int):
    df = dataframe.copy()
    recentAtr = ta.ATR(df["high"], df["low"], df["close"], timeperiod=minLength)
    historicalAtr = ta.ATR(df["high"], df["low"], df["close"], timeperiod=maxLength)
    return recentAtr > historicalAtr

def regime_filter(dataframe, threshold):
    df = dataframe.copy()
    ohlc4 = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    src = ohlc4

    value1 = pd.Series(0,index=df['close'].index, dtype=float)
    value2 = pd.Series(0,index=df['close'].index, dtype=float)
    klmf = pd.Series(0,index=df['close'].index, dtype=float)

    for i in range(0, len(value1)):
        if(i == 0):
            value1[i] = 0
        else:
            value1[i] = 0.2 * (src[i] - src[i-1]) + 0.8 * value1[i-1]

    for i in range(0, len(value1)):
        if(i == 0):
            value2[i] = 0.1 * (df['high'][i] - df['low'][i]) + 0.8 * 0
        else:
            value2[i] = 0.1 * (df['high'][i] - df['low'][i]) + 0.8 * value2[i-1]

    omega = abs(value1 / value2)
    alpha = (-omega**2 + np.sqrt(omega**4 + 16 * omega**2)) / 8 

    for i in range(0, len(value1)):
        if(i == 0):
            klmf[i] = alpha[i] * src[i] + (1 - alpha[i]) * 0
        else:
            klmf[i] = alpha[i] * src[i] + (1 - alpha[i]) * klmf[i-1]

    absCurveSlope = klmf.diff().abs()
    exponentialAverageAbsCurveSlope = 1.0 * ta.EMA(absCurveSlope, 200)
    normalized_slope_decline = (absCurveSlope - exponentialAverageAbsCurveSlope) / exponentialAverageAbsCurveSlope
    return normalized_slope_decline >= threshold

def ema_filter(dataframe,period):
    df = dataframe.copy()
    ema = ta.EMA(df['close'], period)
    filter_value = (df['close'] > ema).astype(int) - (df['close'] < ema).astype(int)
    return filter_value

def sma_filter(dataframe,period):
    df = dataframe.copy()
    sma = ta.SMA(df['close'], period)
    filter_value = (df['close'] > sma).astype(int) - (df['close'] < sma).astype(int)
    return filter_value

def kernel_filter(dataframe,loopback,relative_weight,start_at_bar):
    df = dataframe.copy()
    khat1 = pd.Series(rational_quadratic(df['close'], loopback, relative_weight, start_at_bar))
    # wasBearishRate = khat1.shift(2) > khat1.shift(1)
    # isBearishRate = khat1.shift(1) > khat1
    # wasBullishRate = khat1.shift(2) < khat1.shift(1)
    filter_rate = (khat1.shift(1) < khat1).astype(int) - (khat1.shift(1) > khat1).astype(int)
    return filter_rate

def rational_quadratic(
    price_feed: np.ndarray,
    lookback: int,
    relative_weight: float,
    start_at_bar: int,
) -> np.ndarray:
    length_of_prices = len(price_feed)
    bars_calculated = start_at_bar + 1

    result = np.zeros(length_of_prices, dtype=float)
    lookback_squared = np.power(lookback, 2)
    denominator = lookback_squared * 2 * relative_weight

    for index in range(length_of_prices):
        current_weight = 0.0
        cumulative_weight = 0.0

        for i in range(bars_calculated):
            y = np.nan if (index - i) < 0 else price_feed[index - i]
            w = np.power(
                1 + (np.power(i, 2) / denominator),
                -relative_weight,
            )
            current_weight += y * w
            cumulative_weight += w

        result[index] = current_weight / cumulative_weight

    return result

def gaussian(
    price_feed: np.ndarray,
    lookback: int,
    start_at_bar: int,
) -> np.ndarray:
    length_of_prices = len(price_feed)
    bars_calculated = start_at_bar + 1

    result = np.zeros(length_of_prices, dtype=float)
    lookback_squared = np.power(lookback, 2)
    denominator = lookback_squared * 2

    for index in range(length_of_prices):
        current_weight = 0.0
        cumulative_weight = 0.0

        for i in range(bars_calculated):
            y = np.nan if (index - i) < 0 else price_feed[index - i]
            w = np.exp(-(np.power(i, 2) / denominator))
            current_weight += y * w
            cumulative_weight += w

        result[index] = current_weight / cumulative_weight

    return result

def xATRTrailingStop_func(close, prev_close, prev_atr, nloss):
    if close > prev_atr and prev_close > prev_atr:
        return max(prev_atr, close - nloss)
    elif close < prev_atr and prev_close < prev_atr:
        return min(prev_atr, close + nloss)
    elif close > prev_atr:
        return close - nloss
    else:
        return close + nloss

def calculate_ut_bot(dataframe, SENSITIVITY, ATR_PERIOD):
    df = dataframe.copy()
    # UT Bot Parameters
    SENSITIVITY = SENSITIVITY
    ATR_PERIOD = ATR_PERIOD

    # Compute ATR And nLoss variable
    df["xATR"] = ta.ATR(
        df["high"], df["low"], df["close"], timeperiod=ATR_PERIOD)
    df["nLoss"] = SENSITIVITY * df["xATR"]

    df["ATRTrailingStop"] = [0.0] + \
        [np.nan for i in range(len(df) - 1)]

    for i in range(1, len(df)):
        df.loc[i, "ATRTrailingStop"] = xATRTrailingStop_func(
            df.loc[i, "close"],
            df.loc[i - 1, "close"],
            df.loc[i - 1, "ATRTrailingStop"],
            df.loc[i, "nLoss"],
        )

    # Buy Signal
    df.loc[
        (
            (df["close"] > df["ATRTrailingStop"])
        ),
        'UT_Signal'] = 1

    df.loc[
        (
            (df["close"] < df["ATRTrailingStop"])
        ),
        'UT_Signal'] = -1

    return df['UT_Signal']


class Filter(Enum):
    volatility = "filter_volatility"
    regime = "regime_filter"
    trend = "trend_filter"
    stc = "stc_filter"
    ut = "ut_filter"

In [ ]:
def getLorentizanDistance(i,current_feature,feature_array):
    feature_distance = math.log(1+ abs(current_feature - feature_array[i]))
    return feature_distance

def fractalFilters(predict_value:pd.Series):
    isDifferentSignalType = predict_value.ne(predict_value.shift())
    return isDifferentSignalType

def setPredictionAsClearWay(index,dataframe:pd.DataFrame,filter_method):
    df = dataframe.copy()
    global signal_predictions
    prediction_value = 0
    predicted_value = df['predicted_value'].iloc[index]
    filter_value = True

    buy_trend = True
    sell_trend = True

    buy_stc = True
    sell_stc = True

    buy_ut = True
    sell_ut = True

    for filter in filter_method:
        if(filter.name == "trend"):
            buy_trend = (df[filter.value].iloc[index] > 0)
            sell_trend = (df[filter.value].iloc[index] < 0)
        elif(filter.name == "stc"):
            buy_stc = (df[filter.value].iloc[index] > 0)
            sell_stc = (df[filter.value].iloc[index] < 0)
        elif(filter.name == "ut"):
            buy_ut = (df[filter.value].iloc[index] > 0)
            sell_ut = (df[filter.value].iloc[index] > 0)
        else:
            filter_value = (df[filter.value].iloc[index]) and (filter_value)

    if (predicted_value > 0) & filter_value & buy_trend & buy_stc & buy_ut:
        prediction_value = 1
    elif (predicted_value < 0) & filter_value & sell_trend & sell_stc & sell_ut:
        prediction_value = -1
    else:
        if index == 0:
            prediction_value = 0
        else:
            prediction_value = signal_predictions[index-1]
    signal_predictions[index] = prediction_value
    return prediction_value

def split_predictions_with_proba(current_predict_class,current_predict_probability,dataframe,n_predict_candle) -> dict:
    final_predictions = {}
    df_for_predict = dataframe.copy()

    current_predictions_with_prob = zip(
        current_predict_class, current_predict_probability)

    for i, (pred_class, prob) in enumerate(current_predictions_with_prob, 1):
        predict_minute = (
            df_for_predict.iloc[i-1]['date'].minute + n_predict_candle) % 60
        predict_class = pred_class
        predict_probability = prob

        final_predictions[predict_minute] = {
            "index": i,
            "class": predict_class,
            "probability": predict_probability
        }
    return final_predictions

# def train_model(index,df,training_params):
#     current_index = index
#     lastDistance = -1.0
#     neighbour_count = training_params['neighbor_count']
#     feature_count = training_params['feature_count']
#     # Variable Used for ML
#     global distances
#     global predictions

#     feature_array_1 = df['f1'].to_numpy()
#     feature_array_2 = df['f2'].to_numpy()
#     feature_array_3 = df['f3'].to_numpy()
#     feature_array_4 = df['f4'].to_numpy()
#     feature_array_5 = df['f5'].to_numpy()
#     y_train_array = df['y_train'].to_numpy()

#     current_feature_1 =  feature_array_1[current_index]
#     current_feature_2 =  feature_array_2[current_index]
#     current_feature_3 =  feature_array_3[current_index]
#     current_feature_4 =  feature_array_4[current_index]
#     current_feature_5 =  feature_array_5[current_index] 

#     feature_array_1 = feature_array_1[:current_index+1]
#     feature_array_2 = feature_array_2[:current_index+1]
#     feature_array_3 = feature_array_3[:current_index+1]
#     feature_array_4 = feature_array_4[:current_index+1]
#     feature_array_5 = feature_array_5[:current_index+1]

#     y_train_array = y_train_array[:current_index+1]

#     for i in range(0,current_index+1,1):
#         d = 0
#         current_feature_names = [current_feature_1, current_feature_2,current_feature_3,current_feature_4,current_feature_5]
#         feature_array_names = [feature_array_1, feature_array_2,feature_array_3,feature_array_4,feature_array_5]
#         current_feature_names = current_feature_names[:feature_count]
#         feature_array_names = feature_array_names[:feature_count]

#         for var_index,_ in enumerate(current_feature_names):
#             current_feature_count = current_feature_names[var_index]
#             feature_array_count = feature_array_names[var_index]
#             d = getLorentizanDistance(i,current_feature_count,feature_array_count) + d
        
#         if (d >= lastDistance) and (i%4):
#             lastDistance = d
#             distances.append(d)
#             predictions.append(round(y_train_array[i]))
#             if len(predictions) > neighbour_count:
#                 lastDistance = distances[round(neighbour_count*3/4)]
#                 distances.pop(0)
#                 predictions.pop(0)
    
#     prediction = sum(predictions)
#     return prediction

def train_model(dataframe,training_params):
    # Variable To Use
    feature_count = training_params["feature_count"]
    future_count = training_params["future_count"]

    # Copy Dataframe
    df = dataframe.copy()
    df_for_predict = dataframe.copy()

    # Dropping NAN Value
    df.dropna(inplace=True)
    df_for_predict.fillna(0,inplace=True)

    # Extract Feature List In data_x and data_y for training and predicting
    feature_list = [f'f{i}' for i in range(1, feature_count + 1)]
    data_x = df[feature_list].values
    data_y = df['y_train'].values
    data_for_predict = df_for_predict[feature_list].values

    # Scaling Using StandardScaler
    scaler_for_train = StandardScaler()
    data_x = scaler_for_train.fit_transform(data_x)

    scaler_for_test = StandardScaler()
    data_for_predict = scaler_for_test.fit_transform(data_for_predict)

    # Split Data For Training
    X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=60/num_candles_to_fetch, random_state=42)
    # Build Model
    xgb_model = XGBClassifier(
        tree_method="auto",
    )
    model = xgb_model
    a = datetime.now()
    model.fit(X_train, y_train)
    joblib.dump(model, '../../trained_model.pkl')
    b = datetime.now()
    # Predict Model
    length_to_predict = data_for_predict.shape[0]-future_count
    current_predict_class = model.predict(data_for_predict[:length_to_predict])
    current_predict_class[current_predict_class == 0] = -1
    current_predict_probability = model.predict_proba(data_for_predict[:length_to_predict])
    for _ in range(length_to_predict,length_to_predict+future_count):
        current_predict_class = np.append(current_predict_class,0)
        current_predict_probability = np.vstack([current_predict_probability,[0,0]])
    current_predict_probability = np.amax(current_predict_probability,axis=1,keepdims=True)

    # Testing Model Performance
    y_pred = model.predict(X_test)
    print(f"Accuracy Test : {accuracy_score(y_test,y_pred)}")
    print(f"speed code is {round((b-a).microseconds * 0.001, 4)}ms")
    return current_predict_class,current_predict_probability

def predict_future(dataframe:pd.DataFrame,training_params):
    df = dataframe.copy()
    global signal_predictions
    signal_predictions = {}
    filter_method = training_params['filter_method']

    # df['predicted_value'] = df.apply((lambda x : train_model(x.name,df,training_params)),axis=1)
    df['predicted_value'], df['predicted_proba'] = train_model(df,training_params)
    # df['predicted_value'] = df.apply(lambda x : setPredictionAsClearWay(x.name,df,filter_method),axis=1)
    df['isDifferentSignalType'] = fractalFilters(df['predicted_value'])
    dataframe['predicted_value'] = df['predicted_value']
    dataframe['predicted_proba'] = df['predicted_proba']
    dataframe['buy_signal'] = (df['predicted_value'] > 0) & (df['isDifferentSignalType'])
    dataframe['sell_signal'] = (df['predicted_value'] < 0) & (df['isDifferentSignalType'])
    return dataframe

In [ ]:
async def mlRunModel(dataframe,training_params):
    df = dataframe.copy()
    global dataframe_for_compare
    # heikin_ashi_dataframe = heikinashi(df)

    # trend_periods = training_params['filter_params']['trend']['periods']
    # stc_length = training_params['filter_params']['stc']['length']
    # stc_fast_length = training_params['filter_params']['stc']['fast']
    # stc_slow_length = training_params['filter_params']['stc']['slow']
    # ut_sensitivity = training_params['filter_params']['ut']['sensitivity']
    # ut_atrperiod = training_params['filter_params']['ut']['atr_period']

    df = extract_features(df,training_params)

    # df[Filter.volatility.value] = filter_volatility(df,1,10)
    # df[Filter.regime.value] = regime_filter(df,training_params['filter_params']['regime']['threshold'])
    # df[Filter.trend.value] = adaptiveTrendFinder(heikin_ashi_dataframe,trend_periods)
    # df[Filter.stc.value] = calculateSTCIndicator(df,stc_length,stc_fast_length,stc_slow_length)
    # df[Filter.ut.value] = calculate_ut_bot(heikin_ashi_dataframe,ut_sensitivity,ut_atrperiod)
    # df = predict_future(df,training_params)
    return df

distances = []
predictions = []
signal_predictions = {}
training_params = {
    "filter_method" : [], #done
    "filter_params" : {
        "regime" : {
            "threshold" : -0.1
        },
        "trend":{
            "periods" : [0,5,10]
        },
        "stc":{
            "length" : 12,
            "fast" : 26,
            "slow" : 50
        },
        "ut" : {
            "sensitivity" : 1,
            "atr_period" : 10,
        }
    },
    "is_pca_on" : True,
    "neighbor_count" : 8,
    "feature_count" : 8,
    "future_count" : 1, # 2 means 2 ahead if timeframe = 5m , 5*2 = 10m,if current is 30,then prediction will be 40-45
    "pivot_params" : {
        "depth" : 8,
        "deviation" : 2,
        "backstep" : 2,
    },
    "f1" : {
        "name" : FeatureName.rsi,
        "paramsA" : 9,
        "paramsB" : 2
    },
    "f2" : {
        "name" : FeatureName.rsi,
        "paramsA" : 14,
        "paramsB" : 2
    },
    "f3" : {
        "name" : FeatureName.cci,
        "paramsA" : 20,
        "paramsB" : 2
    },
    "f4" : {
        "name" : FeatureName.adx,
        "paramsA" : 20,
        "paramsB" : 2
    },
    "f5" : {
        "name" : FeatureName.wt,
        "paramsA" : 10,
        "paramsB" : 11
    },
    "f6" : {
        "name" : FeatureName.trend,
        "paramsA" : [0,3,4,10,15],
        "paramsB" : 0
    },
    "f7" : {
        "name" : FeatureName.sma,
        "paramsA" : 11,
        "paramsB" : 2
    },
    "f8" : {
        "name" : FeatureName.stc,
        "paramsA" : "5-10",
        "paramsB" : 2
    }
}

a = datetime.now()
# ml_dataframe = dataframe.copy()
# ml_dataframe.drop(ml_dataframe.tail(1).index, inplace=True)
complete_df = await mlRunModel(dataframe,training_params)
# print(complete_df.tail(20).loc[:,['date','y_train','close']])
# n_predict_candle = training_params['future_count']
# next_predict_df = complete_df[len(complete_df)-n_predict_candle-1:len(complete_df)-1]
# prediction_for_next_mearket = split_predictions_with_proba(next_predict_df['predicted_value'],next_predict_df['predicted_proba'],next_predict_df,n_predict_candle)
# filtered_df = complete_df[(complete_df['buy_signal'] == True) | (complete_df['sell_signal'] == True)]
b = datetime.now()
# dates_with_signals = filtered_df.loc[:,['date','buy_signal','sell_signal']]
# print(prediction_for_next_mearket)
print(f"speed is {round((b-a).microseconds * 0.001, 4)}ms")
# print(complete_df['buy_signal'].value_counts(),complete_df['sell_signal'].value_counts())
# print(dates_wit  h_signals)

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k
        
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_test):
        predictions = []
        for i in range(len(X_test)):
            # distances = np.sum(np.log(1+np.abs(self.X_train - X_test[i])),axis=1)
            distances = np.sqrt(np.sum(np.square(self.X_train - X_test[i]), axis=1))
            nearest_neighbors = np.argsort(distances)[:self.k]
            nearest_labels = [self.y_train[idx] for idx in nearest_neighbors]
            prediction = np.sum(nearest_labels)
            if prediction < -50:
                prediction = -1
            elif prediction > 50:
                prediction = 1
            else:
                prediction = 0
            # prediction = max(set(nearest_labels), key=nearest_labels.count)
            predictions.append(prediction)
        return np.array(predictions)

# Example usage:
a = datetime.now()
feature_count = training_params["feature_count"]
future_count = training_params["future_count"]
is_pca_on = training_params["is_pca_on"]

scaler_for_train = StandardScaler()
feature_list = [f'f{i}' for i in range(1, feature_count + 1)]
knn_dataframe = complete_df.copy()

# Data Preparation
data_x_test = knn_dataframe[feature_list].values
data_x_test = scaler_for_train.fit_transform(data_x_test)
data_y_test = knn_dataframe['y_train'].values
data_y_test_reshaped = data_y_test.reshape(-1, 1)
combination_data = np.hstack((data_x_test, data_y_test_reshaped))
combination_data = combination_data[~np.isnan(combination_data).any(axis=1)]
data_x_test = combination_data[:, :-1]
data_y_test = combination_data[:, -1]
pca = PCA(n_components=1)
if(is_pca_on):
    data_x_test = pca.fit_transform(data_x_test)
# print(knn_dataframe.tail(30).loc[:,['date','y_train','close']])

# # Data Split 
train_size = 100

# data_y_test_reshaped = data_y_test.reshape(-1, 1)
# combination_data = np.hstack((data_x_test, data_y_test_reshaped))
# temp_df = pd.DataFrame(combination_data,columns=feature_list+['y_train'])
# temp_df.dropna(inplace=True)
# data_x = temp_df[feature_list].values
# data_y = temp_df['y_train'].values

X_train = data_x_test[:len(data_x_test)-train_size]
y_train = data_y_test[:len(data_y_test)-train_size]

X_test = data_x_test[len(data_x_test)-train_size:]
y_test = data_y_test[len(data_x_test)-train_size:]

# Fit the model
knn = KNN(k=100)
knn.fit(X_train, y_train)

# knn = CatBoostClassifier(verbose=False,n_estimators=2,learning_rate=0.001)
# knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)
nan_index_of_y_test = np.where(np.isnan(y_test))[0]

index_of_0_in_y_pred = np.where(y_pred == 0)[0]
y_pred = np.delete(y_pred, index_of_0_in_y_pred)
y_test = np.delete(y_test,index_of_0_in_y_pred)
if(len(nan_index_of_y_test) > 0):
    # print("Next Market Move : ",y_pred[nan_index_of_y_test[0]:])
    b = datetime.now()
    print(f"speed is {round((b-a).microseconds * 0.001, 4)}ms")
    print(f"Accuracy Test : {accuracy_score(y_pred[:nan_index_of_y_test[0]],y_test[:nan_index_of_y_test[0]])}")
else:
    # print("Next Market Move : ",y_pred)
    b = datetime.now()
    print(len(y_test))
    print(y_pred)
    # print(index_of_0_in_y_pred)
    print(f"speed is {round((b-a).microseconds * 0.001, 4)}ms")
    print(f"Accuracy Test : {accuracy_score(y_pred,y_test)}")

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# For PCA
# data_x_plot = data_x_test
# data_y_plot = data_y_test

# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')
# x_pca_1 = data_x_plot[data_y_plot == 1][:,0]
# y_pca_1 = data_x_plot[data_y_plot == 1][:,1]
# z_pca_1 = data_x_plot[data_y_plot == 1][:,2]

# x_pca_2 = data_x_plot[data_y_plot == -1][:,0]
# y_pca_2 = data_x_plot[data_y_plot == -1][:,1]
# z_pca_2 = data_x_plot[data_y_plot == -1][:,2]
# # Scatter plot
# ax.scatter(x_pca_1,y_pca_1, z_pca_1, c='green', marker='o', alpha=0.5)
# ax.scatter(x_pca_2,y_pca_2, z_pca_2, c='red', marker='o', alpha=0.5)
# # Set labels and title
# ax.set_xlabel('PCA Component 1')
# ax.set_ylabel('PCA Component 2')
# ax.set_zlabel('PCA Component 3')
# ax.set_title('PCA Visualization of Training Data')

# plt.show()

# Create trace for each class
# trace1 = go.Scatter3d(
#     x=X_train[y_train == 1][:, 0],
#     y=X_train[y_train == 1][:, 1],
#     z=X_train[y_train == 1][:, 2],
#     mode='markers',
#     marker=dict(
#         size=5,
#         color='green',  # set color to an array/list of desired values
#         opacity=0.5
#     ),
#     name='Class 1'
# )

# trace2 = go.Scatter3d(
#     x=X_train[y_train == -1][:, 0],
#     y=X_train[y_train == -1][:, 1],
#     z=X_train[y_train == -1][:, 2],
#     mode='markers',
#     marker=dict(
#         size=5,
#         color='red',  # set color to an array/list of desired values
#         opacity=0.5
#     ),
#     name='Class -1'
# )

# # Create layout
# layout = go.Layout(
#     scene=dict(
#         xaxis=dict(title='PCA Component 1'),
#         yaxis=dict(title='PCA Component 2'),
#         zaxis=dict(title='PCA Component 3')
#     ),
#     title='PCA Visualization of Training Data'
# )

# # Create figure
# fig = go.Figure(data=[trace1, trace2], layout=layout)

# # Show plot
# fig.show()